<a href="https://colab.research.google.com/github/zelal-Eizaldeen/project2_Walmart_Price_Prediction/blob/main/Project_1_3113_zelalae2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###########################################################
######### to do:
######### prediction based on log / remember to translate back
######### need to name the code as mymain.py ########
######### replace the csv files links
######### need to wrap the whole model in function and loop through folder 1-10
######### test in Python 3.11.10
######### split report in its own file ########
######### take out the loop #########
###########################################################

# **Team Members**

Zilal Eiz Aldin (Online MCS) Zelalae2@illinois.edu

Mesay Taye (Online MCS) mesayst2@illinois.edu

Jianci Zhai (Online MCS) jianciz2@illinois.edu

# **Contribution of each member**

xxx By Mesay

Data Preparation By Zilal

Linear Model with  Elasticnet Penalty / Boosting Tree By Jianci

# **Project Related Links:**
https://campuswire.com/c/GB46E5679/feed/273

https://liangfgithub.github.io/Proj/F24_Proj1.pdf

https://campuswire.com/c/GB46E5679/feed/329

# **Basic Setting**

Python 3.11.10
catboost==1.2.7
category-encoders==2.6.3
feature-engine==1.8.1
fonttools==4.54.1
glmnet-py==0.1.0b2
lightgbm==4.5.0
numpy==1.26.4
pandas==2.2.3
rpy2==3.5.16
scikit-learn==1.5.2
scipy==1.14.1
xgboost==2.1.1

In [ ]:
!python --version
#Python 3.10.12, need to re-test code in 3.11.10

Python 3.10.12


Step 0: Load Necesary Python Packages

In [ ]:
!pip install catboost==1.2.7
!pip install category-encoders==2.6.3
!pip install feature-engine==1.8.1
!pip install fonttools==4.54.1
!pip install glmnet-py==0.1.0b2
!pip install lightgbm==4.5.0
!pip install numpy==1.26.4
!pip install pandas==2.2.3
!pip install rpy2==3.5.16
!pip install scikit-learn==1.5.2
!pip install scipy==1.14.1
!pip install xgboost==2.1.1

#remove after finish
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.1/364.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.3/378.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 86.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.6.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
     ━━

In [ ]:
# import all libraries
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import scale
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error,make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb


import warnings # supress warnings
warnings.filterwarnings('ignore')

In [ ]:
# remove before submitting
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Loop through fold1 to fold10
# Main function to handle all folds
for fold in range(1, 11):
    train = pd.read_csv(f'/content/drive/My Drive/cs598_project_file/proj1/fold{fold}/train.csv')
    test = pd.read_csv(f'/content/drive/My Drive/cs598_project_file/proj1/fold{fold}/test.csv')

    train=train.set_index('PID')
    test=test.set_index('PID')



    train_y = train['Sale_Price']
    train = train.drop(columns=['Sale_Price'])

    df_shape = train.shape
    df_shape

    # basement related features.
    # missing values are likely zero for having no basement
    # reference: https://www.kaggle.com/code/ahmedabdellahismail/house-prices-advanced-regression-techniques
    for col in ('BsmtFin_SF_1', 'BsmtFin_SF_2', 'Bsmt_Unf_SF','Total_Bsmt_SF', 'Bsmt_Full_Bath', 'Bsmt_Half_Bath'):
      train[col].fillna(0,inplace=True)
      test[col].fillna(0,inplace=True)


    for col in ('Bsmt_Qual', 'Bsmt_Cond', 'Bsmt_Exposure', 'BsmtFin_Type_1', 'BsmtFin_Type_2'):
        train[col].fillna('None',inplace=True)
        test[col].fillna('None',inplace=True)



    #convert important cat varaibles to ord var
    #need to replace with a function :p
    category_order = ['Very_Excellent','Excellent','Very_Good','Good','Above_Average','Average','Below_Average','Typical','None','Poor','Very_Poor','Fair']
    #category_order = ['Very_Excellent','Excellent','Very_Good','Good','Above_Average','Average','Below_Average','Poor','Very_Poor']


    for col in ('Overall_Qual','Overall_Cond','Bsmt_Qual', 'Bsmt_Cond','Kitchen_Qual','Exter_Qual','Exter_Cond','Heating_QC','Fireplace_Qu', 'Garage_Qual','Garage_Cond','Pool_QC'):
        train[col].fillna('None',inplace=True)
        test[col].fillna('None',inplace=True)

        train[col] = pd.Categorical(train[col], categories=category_order, ordered=True)
        train[col] = train[col].cat.codes.astype('int64')

        test[col] = pd.Categorical(test[col], categories=category_order, ordered=True)
        test[col] = test[col].cat.codes.astype('int64')


    regular_order = ['Irregular','Moderately_Irregular','Slightly_Irregular','Regular']

    train['Lot_Shape'] = pd.Categorical(train['Lot_Shape'], categories=regular_order, ordered=True)
    train['Lot_Shape'] = train['Lot_Shape'].cat.codes.astype('int64')

    test['Lot_Shape'] = pd.Categorical(test['Lot_Shape'], categories=regular_order, ordered=True)
    test['Lot_Shape'] = test['Lot_Shape'].cat.codes.astype('int64')

    #cap/cat important variables
    train['TotRms_AbvGrd'] = np.where(train['TotRms_AbvGrd']>8,9,train['TotRms_AbvGrd'])
    test['TotRms_AbvGrd'] = np.where(test['TotRms_AbvGrd']>8,9,test['TotRms_AbvGrd'])

    train['Mo_Sold'] = np.where(train['Mo_Sold'].isin([5,6,7]),'5-7','other mo')
    test['Mo_Sold'] = np.where(test['Mo_Sold'].isin([5,6,7]),'5-7','other mo')

    train['MS_Zoning'] = np.where(train['MS_Zoning'].isin(['C_all','Floating_Village_Residential','Residential_High_Density','Residential_Low_Density','Residential_Medium_Density',]),train['MS_Zoning'],'other')
    test['MS_Zoning'] = np.where(test['MS_Zoning'].isin(['C_all','Floating_Village_Residential','Residential_High_Density','Residential_Low_Density','Residential_Medium_Density',]),test['MS_Zoning'],'other')

    train['Condition_1'] = np.where(train['Condition_1'].isin(['Artery','Feedr','Norm','PosA','PosN']),train['Condition_1'],'other')
    test['Condition_1'] = np.where(test['Condition_1'].isin(['Artery','Feedr','Norm','PosA','PosN']),test['Condition_1'],'other')


    train['Condition_1'] = np.where(train['Condition_1'].isin(['Norm']),train['Condition_1'],'other')
    test['Condition_1'] = np.where(test['Condition_1'].isin(['Norm']),test['Condition_1'],'other')

    train['Exterior_1st'] = np.where(train['Exterior_1st'].isin(['BrkFace']),train['Exterior_1st'],'other')
    test['Exterior_1st'] = np.where(test['Exterior_1st'].isin(['BrkFace']),test['Exterior_1st'],'other')


    train['MS_SubClass'] = np.where(train['MS_SubClass'].isin(['Duplex_All_Styles_and_Ages'
                              ,'One_Story_1945_and_Older','One_Story_1946_and_Newer_All_Styles','One_Story_PUD_1946_and_Newer'
                              ,'One_and_Half_Story_Finished_All_Ages','Split_or_Multilevel','Two_Story_1945_and_Older'
                              ,'Two_Story_1946_and_Newer','Two_Story_PUD_1946_and_Newer']),train['MS_SubClass'],'other')
    test['MS_SubClass'] = np.where(test['MS_SubClass'].isin(['Duplex_All_Styles_and_Ages'
                              ,'One_Story_1945_and_Older','One_Story_1946_and_Newer_All_Styles','One_Story_PUD_1946_and_Newer'
                              ,'One_and_Half_Story_Finished_All_Ages','Split_or_Multilevel','Two_Story_1945_and_Older'
                              ,'Two_Story_1946_and_Newer','Two_Story_PUD_1946_and_Newer']),test['MS_SubClass'],'other')



    train['Neighborhood'] = np.where(train['Neighborhood'].isin(['North_Ames','College_Creek','Old_Town',
                                                                  'Somerset','Edwards','Northridge_Heights','Crawford','Iowa_DOT_and_Rail_Road',
                                                                  'Northridge','Timberland','South_and_West_of_Iowa_State_University','Stone_Brook',
                                                                  'Clear_Creek','Meadow_Village']),train['Neighborhood'],'other')
    test['Neighborhood'] = np.where(test['Neighborhood'].isin(['North_Ames','College_Creek','Old_Town',
                                                                  'Somerset','Edwards','Northridge_Heights','Crawford','Iowa_DOT_and_Rail_Road',
                                                                  'Northridge','Timberland','South_and_West_of_Iowa_State_University','Stone_Brook',
                                                                  'Clear_Creek','Meadow_Village']),test['Neighborhood'],'other')

    # first fill na with 0 and will come back later
    train.fillna(0, inplace=True)
    test.fillna(0, inplace=True)


    #month and year should both be categorical
    train['Mo_Sold']=train['Mo_Sold'].astype('object')
    train['Year_Sold']=train['Year_Sold'].astype('object')

    test['Mo_Sold']=test['Mo_Sold'].astype('object')
    test['Year_Sold']=test['Year_Sold'].astype('object')

    #exclude longtitude and lattitude based on initial feature importance
    train = train.drop(columns=['Longitude','Latitude'])
    #test = test.drop(columns=['Longitude','Latitude','Roof_Style','Exterior_2nd','Garage_Finish','Fence','BsmtFin_Type_2','Sale_Type','Alley'])



    train.head()

    train.describe()

    df_dict = pd.DataFrame({
        'Variable':train.columns.tolist(),
        'Type':train.dtypes.tolist(),
        'Unique_Values':train.nunique().tolist(),
        'Missing_Values':train.isnull().sum().tolist()
    })

    # for quick processing, data type in ('int64','float64') assume to be continuous
    # data type in ['object'] assume to be categorical
    # if categorical - unique_values > 20 - this rule is pretty objective, will review later

    # check the skewness of conti. variables

    cont_var = df_dict[(df_dict['Type']=='int64') | (df_dict['Type']=='float64')]['Variable'].unique().tolist()
    cont_var

    df_dict['skewness'] = np.NAN
    df_dict['skewness_type'] = np.NAN
    for var in cont_var:
      skew =  train[var].skew()
      df_dict.loc[df_dict[df_dict['Variable']==var].index,'skewness'] = skew
      if abs(skew) <= 3:
        df_dict.loc[df_dict[df_dict['Variable']==var].index,'skewness_type'] = 1
      #print(f'{var} with skewness = {skew}')

    df_dict

    #good conitnues variable should with a skew between -1 and 1, ideally the unique values should be big

    cont_var_good = df_dict[(df_dict['skewness_type']==1) & (df_dict['Unique_Values'] > 1)]['Variable'].unique().tolist()
    cont_var_good

    #catigorical variables
    #cat_var = df_dict[(df_dict['Type']=='object')|((df_dict['Type']=='int64') & (df_dict['Unique_Values'] <= 20))]['Variable'].unique().tolist()
    cat_var = df_dict[(df_dict['Type']=='object')]['Variable'].unique().tolist()
    cat_var

    try:
      del var_dt_summary
    except:
      print('no var_dt_sumary created yet')
    for i, var in enumerate(cat_var):
      #print(var)
      var_dt = train.groupby(var)[var].count().to_frame()
      var_dt.columns = ['Count']
      var_dt = var_dt.reset_index()
      var_dt.columns = ['Value','Count']
      var_dt['Variable'] = var

      try:
        var_dt_summary=pd.concat([var_dt_summary,var_dt], axis=0)
      except:
        var_dt_summary=var_dt

    var_dt_summary=var_dt_summary[['Variable','Value','Count']]



    high_card_cutoff = 50

    cat_var_high_card = df_dict[(df_dict['Variable'].isin(cat_var)) & (df_dict['Unique_Values']>= high_card_cutoff)]['Variable'].unique().tolist()
    cat_var_high_card

    imbalance_cutoff = 0.999 #if a count of a variable > 70% of total then it's inbalance

    cat_var_imb = var_dt_summary[var_dt_summary['Count']> (imbalance_cutoff * df_shape[0])]['Variable'].unique().tolist()
    cat_var_imb

    # for model only use balanced & low-card categorical variables

    cat_var_good = [item for item in cat_var if (item not in cat_var_high_card) and (item not in cat_var_imb)]
    cat_var_good
    df_dict['Selected'] = np.where((df_dict['Variable'].isin(cat_var_good)|(df_dict['Variable'].isin(cont_var_good))), True, False)


    # Subset all categorical variables
    def convert_cat_vars(df,cat_var_good):
      df_categorical = df[cat_var_good]
      # convert into dummies
      df_dummies = pd.get_dummies(df_categorical, columns = cat_var_good , drop_first=True)

      return df_dummies

    train_cat = convert_cat_vars(train,cat_var_good)
    train_cat

    test_cat = convert_cat_vars(test,cat_var_good)
    test_cat

    """Scale and winsorize data"""

    def convert_cont_vars(df,cat_var_good):
      df_cont = df[cont_var_good]
      # convert into dummies
      df_wzr = df_cont.clip(upper=df_cont.quantile(0.9854518181818182),axis=1)
      scaler = StandardScaler()
      df_scaled = pd.DataFrame(scaler.fit_transform(df_wzr), columns=df_wzr.columns, index=df_wzr.index)

      return df_scaled

    train_cont = convert_cont_vars(train,cont_var_good)
    train_cont

    test_cont = convert_cont_vars(test,cont_var_good)
    test_cont

    train_new = pd.concat([train_cat,train_cont], axis = 1)
    train_new

    test_new = pd.concat([test_cat,test_cont], axis = 1)
    test_new

    train_y_log = np.log(train_y)
    train_y_log = train_y_log.clip(lower=train_y_log.quantile(0.01), upper=train_y_log.quantile(0.99))

    # make sure all columns available in test
    common_columns = test_new.columns.intersection(train_new.columns)


    elastic_net_cv = ElasticNetCV(
                          cv=100,
                          random_state=3113,
                          max_iter=10000)


    #elastic_net_cv = ElasticNetCV(cv=10, random_state=42)

    # Fit the model on training data
    elastic_net_cv.fit(train_new[common_columns], train_y_log)
    start_time = time.time()  # Start timer
    # Best alpha and l1_ratio found via cross-validation
    #print(f"Best alpha: {elastic_net_cv.alpha_}")
    #print(f"Best l1_ratio: {elastic_net_cv.l1_ratio_}")

    # Make predictions on test data
    y_pred_cv = elastic_net_cv.predict(test_new[common_columns])

    end_time = time.time()  # End timer
    print(f"Fold {fold}: Time taken = {end_time - start_time} seconds")

    # Check model performance
    test_y = pd.read_csv(f'/content/drive/My Drive/cs598_project_file/proj1/fold{fold}/test_y.csv')
    mse_cv = mean_squared_error(np.log(test_y.set_index('PID')), y_pred_cv)
    mse_cv
    rmse_cv = np.sqrt(mse_cv)

    coefficients = elastic_net_cv.coef_
    coefficients
    feature_importance = pd.DataFrame({
        'Feature': common_columns,
        'Coefficient': coefficients
    })
    feature_importance

    y_pred = train_y.copy()

    y_pred['Sale_Price'] = np.exp(y_pred_cv)
    y_pred.to_csv(f'/content/drive/My Drive/cs598_project_file/proj1/fold{fold}/mysubmission1.txt', sep=',', index=False)
    df_dict.to_csv(f'/content/drive/My Drive/cs598_project_file/proj1/fold{fold}/df_dict.csv')
    var_dt_summary.to_csv(f'/content/drive/My Drive/cs598_project_file/proj1/fold{fold}/var_dt_summary.csv')
    feature_importance.to_csv(f'/content/drive/My Drive/cs598_project_file/proj1/fold{fold}/feature_importance.csv')

    import xlsxwriter
    with pd.ExcelWriter(f'/content/drive/My Drive/cs598_project_file/proj1/fold{fold}/linear_model_summary.xlsx', engine='xlsxwriter') as writer:
      df_dict.to_excel(writer, sheet_name='df_dict', index=False)
      var_dt_summary.to_excel(writer, sheet_name='var_dt_summary', index=False)
      feature_importance.to_excel(writer, sheet_name='feature_importance', index=False)

    print(f'fold{fold}:Best alpha: {elastic_net_cv.alpha_}; rmse_cv={round(rmse_cv,3)}')

no var_dt_sumary created yet
Fold 1: Time taken = 0.004606962203979492 seconds
fold1:Best alpha: 0.0007950425525360286; rmse_cv=0.121
Fold 2: Time taken = 0.009496212005615234 seconds
fold2:Best alpha: 0.0006502641899394538; rmse_cv=0.12
Fold 3: Time taken = 0.00609588623046875 seconds
fold3:Best alpha: 0.0006589464921850694; rmse_cv=0.117
Fold 4: Time taken = 0.009948253631591797 seconds
fold4:Best alpha: 0.0006213655434303285; rmse_cv=0.123
Fold 5: Time taken = 0.0045392513275146484 seconds
fold5:Best alpha: 0.0006614629861972163; rmse_cv=0.113
Fold 6: Time taken = 0.0051076412200927734 seconds
fold6:Best alpha: 0.0007932683584291237; rmse_cv=0.133
Fold 7: Time taken = 0.005580902099609375 seconds
fold7:Best alpha: 0.0006484218995514943; rmse_cv=0.132
Fold 8: Time taken = 0.0049915313720703125 seconds
fold8:Best alpha: 0.0007049065885589962; rmse_cv=0.13
Fold 9: Time taken = 0.005408525466918945 seconds
fold9:Best alpha: 0.0006202641522285581; rmse_cv=0.134
Fold 10: Time taken = 0.00

In [ ]:
# Loop through fold1 to fold10
# Main function to handle all folds
for fold in range(1, 11):
    train = pd.read_csv(f'/content/drive/My Drive/cs598_project_file/proj1/fold{fold}/train.csv')
    test = pd.read_csv(f'/content/drive/My Drive/cs598_project_file/proj1/fold{fold}/test.csv')

    train=train.set_index('PID')
    test=test.set_index('PID')



    train_y = train['Sale_Price']
    train = train.drop(columns=['Sale_Price'])

    df_shape = train.shape
    df_shape

    # basement related features.
    # missing values are likely zero for having no basement
    # reference: https://www.kaggle.com/code/ahmedabdellahismail/house-prices-advanced-regression-techniques
    for col in ('BsmtFin_SF_1', 'BsmtFin_SF_2', 'Bsmt_Unf_SF','Total_Bsmt_SF', 'Bsmt_Full_Bath', 'Bsmt_Half_Bath'):
      train[col].fillna(0,inplace=True)
      test[col].fillna(0,inplace=True)


    for col in ('Bsmt_Qual', 'Bsmt_Cond', 'Bsmt_Exposure', 'BsmtFin_Type_1', 'BsmtFin_Type_2'):
        train[col].fillna('None',inplace=True)
        test[col].fillna('None',inplace=True)



    #convert important cat varaibles to ord var
    #need to replace with a function :p
    category_order = ['Very_Excellent','Excellent','Very_Good','Good','Above_Average','Average','Below_Average','Typical','None','Poor','Very_Poor','Fair']
    #category_order = ['Very_Excellent','Excellent','Very_Good','Good','Above_Average','Average','Below_Average','Poor','Very_Poor']


    for col in ('Overall_Qual','Overall_Cond','Bsmt_Qual', 'Bsmt_Cond','Kitchen_Qual','Exter_Qual','Exter_Cond','Heating_QC','Fireplace_Qu', 'Garage_Qual','Garage_Cond','Pool_QC'):
        train[col].fillna('None',inplace=True)
        test[col].fillna('None',inplace=True)

        train[col] = pd.Categorical(train[col], categories=category_order, ordered=True)
        train[col] = train[col].cat.codes.astype('int64')

        test[col] = pd.Categorical(test[col], categories=category_order, ordered=True)
        test[col] = test[col].cat.codes.astype('int64')


    regular_order = ['Irregular','Moderately_Irregular','Slightly_Irregular','Regular']

    train['Lot_Shape'] = pd.Categorical(train['Lot_Shape'], categories=regular_order, ordered=True)
    train['Lot_Shape'] = train['Lot_Shape'].cat.codes.astype('int64')

    test['Lot_Shape'] = pd.Categorical(test['Lot_Shape'], categories=regular_order, ordered=True)
    test['Lot_Shape'] = test['Lot_Shape'].cat.codes.astype('int64')

    train.fillna(0, inplace=True)
    test.fillna(0, inplace=True)


    train.head()

    train.describe()

    df_dict = pd.DataFrame({
        'Variable':train.columns.tolist(),
        'Type':train.dtypes.tolist(),
        'Unique_Values':train.nunique().tolist(),
        'Missing_Values':train.isnull().sum().tolist()
    })

    # for quick processing, data type in ('int64','float64') assume to be continuous
    # data type in ['object'] assume to be categorical
    # if categorical - unique_values > 20 - this rule is pretty objective, will review later

    # check the skewness of conti. variables

    cont_var = df_dict[(df_dict['Type']=='int64') | (df_dict['Type']=='float64')]['Variable'].unique().tolist()
    cont_var

    df_dict['skewness'] = np.NAN
    df_dict['skewness_type'] = np.NAN
    for var in cont_var:
      skew =  train[var].skew()
      df_dict.loc[df_dict[df_dict['Variable']==var].index,'skewness'] = skew
      if abs(skew) <= 300:
        df_dict.loc[df_dict[df_dict['Variable']==var].index,'skewness_type'] = 1
      #print(f'{var} with skewness = {skew}')

    df_dict

    #good conitnues variable should with a skew between -1 and 1, ideally the unique values should be big

    cont_var_good = df_dict[(df_dict['skewness_type']==1) & (df_dict['Unique_Values'] > 1)]['Variable'].unique().tolist()
    cont_var_good

    #catigorical variables
    #cat_var = df_dict[(df_dict['Type']=='object')|((df_dict['Type']=='int64') & (df_dict['Unique_Values'] <= 20))]['Variable'].unique().tolist()
    cat_var = df_dict[(df_dict['Type']=='object')]['Variable'].unique().tolist()
    cat_var

    try:
      del var_dt_summary
    except:
      print('no var_dt_sumary created yet')
    for i, var in enumerate(cat_var):
      #print(var)
      var_dt = train.groupby(var)[var].count().to_frame()
      var_dt.columns = ['Count']
      var_dt = var_dt.reset_index()
      var_dt.columns = ['Value','Count']
      var_dt['Variable'] = var

      try:
        var_dt_summary=pd.concat([var_dt_summary,var_dt], axis=0)
      except:
        var_dt_summary=var_dt

    var_dt_summary=var_dt_summary[['Variable','Value','Count']]



    high_card_cutoff = 25

    cat_var_high_card = df_dict[(df_dict['Variable'].isin(cat_var)) & (df_dict['Unique_Values']>= high_card_cutoff)]['Variable'].unique().tolist()
    cat_var_high_card

    imbalance_cutoff = 0.999 #if a count of a variable > 70% of total then it's inbalance

    cat_var_imb = var_dt_summary[var_dt_summary['Count']> (imbalance_cutoff * df_shape[0])]['Variable'].unique().tolist()
    cat_var_imb

    # for model only use balanced & low-card categorical variables

    cat_var_good = [item for item in cat_var if (item not in cat_var_high_card) and (item not in cat_var_imb)]
    cat_var_good
    df_dict['Selected'] = np.where((df_dict['Variable'].isin(cat_var_good)|(df_dict['Variable'].isin(cont_var_good))), True, False)


    # Subset all categorical variables
    def convert_cat_vars(df,cat_var_good):
      df_categorical = df[cat_var_good]
      # convert into dummies
      df_dummies = pd.get_dummies(df_categorical, columns = cat_var_good , drop_first=True)

      return df_dummies

    train_cat = convert_cat_vars(train,cat_var_good)
    train_cat

    test_cat = convert_cat_vars(test,cat_var_good)
    test_cat

    """Scale and winsorize data"""


    def convert_cont_vars(df,cat_var_good, clip_default = 99999):
      df_cont = df[cont_var_good]
      # convert into dummies
      clip = df_cont.quantile(0.99999999)
      df_wzr = df_cont.clip(upper=clip,axis=1)
      scaler = StandardScaler()
      df_scaled = pd.DataFrame(scaler.fit_transform(df_wzr), columns=df_wzr.columns, index=df_wzr.index)

      return df_scaled, clip

    train_cont, clip = convert_cont_vars(train,cont_var_good)
    train_cont

    test_cont, _ = convert_cont_vars(test,cont_var_good)
    test_cont

    train_new = pd.concat([train_cat,train_cont], axis = 1)
    train_new

    test_new = pd.concat([test_cat,test_cont], axis = 1)
    test_new

    train_y_log = np.log(train_y)
    #train_y_log = train_y_log.clip(lower=train_y_log.quantile(0.01), upper=train_y_log.quantile(0.99))

    common_columns = test_new.columns.intersection(train_new.columns)

    start_time = time.time()  # Start timer

    # Initialize the model

    gb = GradientBoostingRegressor(n_estimators=500,random_state=42)

    #grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')
    # Fit the model on training data
    #grid_search.fit(train_new[common_columns], train_y_log)


    gb.fit(train_new[common_columns], train_y_log)

    end_time = time.time()  # End timer
    print(f"Fold {fold}: Time taken = {end_time - start_time} seconds")

    # Make predictions on test data
    y_pred_tree = gb.predict(test_new[common_columns])

    # Check model performance
    test_y = pd.read_csv(f'/content/drive/My Drive/cs598_project_file/proj1/fold{fold}/test_y.csv')
    mse_tree = mean_squared_error(np.log(test_y.set_index('PID')), y_pred_tree)
    rmse_tree = np.sqrt(mse_tree)

    #print(rmse_tree)

    coefficients = gb.feature_importances_
    feature_importance = pd.DataFrame({
        'Feature': common_columns,
        'Coefficient': coefficients})
    #print(feature_importance)

    y_pred = train_y.copy()

    y_pred['Sale_Price'] = np.exp(y_pred_tree)

    y_pred.to_csv(f'/content/drive/My Drive/cs598_project_file/proj1/fold{fold}/mysubmission2.txt', sep=',', index=False)
    df_dict.to_csv(f'/content/drive/My Drive/cs598_project_file/proj1/fold{fold}/df_dict2.csv')
    var_dt_summary.to_csv(f'/content/drive/My Drive/cs598_project_file/proj1/fold{fold}/var_dt_summary2.csv')
    feature_importance.to_csv(f'/content/drive/My Drive/cs598_project_file/proj1/fold{fold}/feature_importance2.csv')

    import xlsxwriter
    with pd.ExcelWriter(f'/content/drive/My Drive/cs598_project_file/proj1/fold{fold}/gb_model_summary.xlsx', engine='xlsxwriter') as writer:
      df_dict.to_excel(writer, sheet_name='df_dict', index=False)
      var_dt_summary.to_excel(writer, sheet_name='var_dt_summary', index=False)
      feature_importance.to_excel(writer, sheet_name='feature_importance', index=False)

    print(f'fold{fold}: rmse_tree = {round(rmse_tree,3)}; ')
#fold10: rmse_tree = 0.14148801043144787
#performance of random forest with all variables in
    #200:fold10: rmse_tree = 0.14138076045921408;fold10: rmse_tree = 0.14126320115166074; fold10: rmse_tree = 0.14100211881142555; take out manual combo of neibouhood
    # with grid search:0.1427801081186383
    # without grid search: 0.1407715048272254


Fold 1: Time taken = 8.420510530471802 seconds
fold1: rmse_tree = 0.117; 
Fold 2: Time taken = 9.8500816822052 seconds
fold2: rmse_tree = 0.124; 
Fold 3: Time taken = 9.63184404373169 seconds
fold3: rmse_tree = 0.121; 
Fold 4: Time taken = 9.421733856201172 seconds
fold4: rmse_tree = 0.111; 
Fold 5: Time taken = 8.37133264541626 seconds
fold5: rmse_tree = 0.115; 
Fold 6: Time taken = 8.879836797714233 seconds
fold6: rmse_tree = 0.13; 
Fold 7: Time taken = 11.080754518508911 seconds
fold7: rmse_tree = 0.134; 
Fold 8: Time taken = 9.831457614898682 seconds
fold8: rmse_tree = 0.131; 
Fold 9: Time taken = 9.483762741088867 seconds
fold9: rmse_tree = 0.127; 
Fold 10: Time taken = 8.398765325546265 seconds
fold10: rmse_tree = 0.124; 


# **=========================== CODE ARCHIVE ======================**

In [ ]:
train_cat.columns

Index(['MS_SubClass_One_Story_1945_and_Older',
       'MS_SubClass_One_Story_1946_and_Newer_All_Styles',
       'MS_SubClass_One_Story_PUD_1946_and_Newer',
       'MS_SubClass_One_Story_with_Finished_Attic_All_Ages',
       'MS_SubClass_One_and_Half_Story_Finished_All_Ages',
       'MS_SubClass_One_and_Half_Story_Unfinished_All_Ages',
       'MS_SubClass_PUD_Multilevel_Split_Level_Foyer',
       'MS_SubClass_Split_Foyer', 'MS_SubClass_Split_or_Multilevel',
       'MS_SubClass_Two_Family_conversion_All_Styles_and_Ages',
       ...
       'Sale_Type_ConLw', 'Sale_Type_New', 'Sale_Type_Oth', 'Sale_Type_VWD',
       'Sale_Type_WD ', 'Sale_Condition_AdjLand', 'Sale_Condition_Alloca',
       'Sale_Condition_Family', 'Sale_Condition_Normal',
       'Sale_Condition_Partial'],
      dtype='object', length=241)

In [ ]:
matching_columns = [col for col in train_cat.columns if col.startswith('Lot_Shape')]
matching_columns

['Lot_Shape_Moderately_Irregular',
 'Lot_Shape_Regular',
 'Lot_Shape_Slightly_Irregular']

In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor(max_leaf_nodes=5)


In [ ]:
train_cat[matching_columns]

,Lot_Shape_Moderately_Irregular,Lot_Shape_Regular,Lot_Shape_Slightly_Irregular
PID,,,
526301100,False,False,True
526351010,False,False,True
527105010,False,False,True
527105030,False,False,True
527145080,False,False,True
...,...,...,...
923230040,False,True,False
923250060,False,True,False
923251180,False,True,False


In [ ]:
tree.fit(train_cat[matching_columns], train_y_log)

DecisionTreeRegressor(max_leaf_nodes=5)

In [ ]:
def rmse(y_true, y_pred):
    #print(y_true, y_pred)
    print(np.sqrt(mean_squared_error(y_true, y_pred)))
    return np.sqrt(mean_squared_error(y_true, y_pred))
rmse_scorer =  make_scorer(rmse, greater_is_better=False)

In [ ]:
alpha_range = np.linspace(0, 0.1, num=5)
alpha_range.tolist()

[0.0, 0.025, 0.05, 0.07500000000000001, 0.1]

In [ ]:
l1_ratio = np.linspace(0, 0.1, num=5)

In [ ]:
param_grid = {
    'alpha': alpha_range.tolist(),       # Strength of the regularization (larger values for stronger regularization)
    'l1_ratio': l1_ratio.tolist()  # Ratio between L1 and L2 regularization (0 = Ridge, 1 = Lasso)
}

In [ ]:
model = ElasticNet()

In [ ]:
train.shape

(2051, 81)

In [ ]:
grid = GridSearchCV(model, param_grid, scoring=rmse_scorer, cv=5, verbose=2)
grid.fit(train_new[common_columns], train_y_log)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
0.17162256905442283
[CV] END ............................alpha=0.0, l1_ratio=0.0; total time=   0.4s
0.16732920406065463
[CV] END ............................alpha=0.0, l1_ratio=0.0; total time=   0.4s
0.1541552126732998
[CV] END ............................alpha=0.0, l1_ratio=0.0; total time=   0.4s
0.15265728130774195
[CV] END ............................alpha=0.0, l1_ratio=0.0; total time=   0.4s
0.13585562075931895
[CV] END ............................alpha=0.0, l1_ratio=0.0; total time=   0.4s
0.17162256905442283
[CV] END ..........................alpha=0.0, l1_ratio=0.025; total time=   0.4s
0.16732920406065463
[CV] END ..........................alpha=0.0, l1_ratio=0.025; total time=   0.4s
0.1541552126732998
[CV] END ..........................alpha=0.0, l1_ratio=0.025; total time=   0.4s
0.15265728130774195
[CV] END ..........................alpha=0.0, l1_ratio=0.025; total time=   0.4s
0.13585562075931895
[CV] END ..

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.0, 0.025, 0.05, 0.07500000000000001, 0.1],
                         'l1_ratio': [0.0, 0.025, 0.05, 0.07500000000000001,
                                      0.1]},
             scoring=make_scorer(rmse, greater_is_better=False, response_method='predict'),
             verbose=2)

In [ ]:
y_pred_cv = grid.predict(test_new[common_columns])

In [ ]:
mse_cv = mean_squared_error(np.log(test_y.set_index('PID')), y_pred_cv)
rmse_cv = np.sqrt(mse_cv)

In [ ]:
rmse_cv

0.15219599993397404

In [ ]:
# Step 6: Check the best parameters and best score
print("Best Parameters: ", grid.best_params_)
print("Best Score: ", grid.best_score_)

Best Parameters:  {'alpha': 0.025, 'l1_ratio': 0.0}
Best Score:  -0.15604701221391037


In [ ]:
test_y

,PID,Sale_Price
0,910251050,13100
1,903454090,141000
2,528253010,207500
3,527183060,188500
4,907135180,123600
...,...,...
874,914476080,100000
875,533206040,176400
876,909281080,257500
877,902105130,64000


In [ ]:
train=train.set_index('PID')
test=test.set_index('PID')


KeyError: "None of ['PID'] are in the columns"

In [ ]:
# first fill na with 0 and will come back later
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [ ]:
train_y = train['Sale_Price']
train = train.drop(columns=['Sale_Price'])

In [ ]:
df_shape = train.shape
df_shape

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
df_dict = pd.DataFrame({
    'Variable':train.columns.tolist(),
    'Type':train.dtypes.tolist(),
    'Unique_Values':train.nunique().tolist(),
    'Missing_Values':train.isnull().sum().tolist()
})

In [ ]:
# for quick processing, data type in ('int64','float64') assume to be continuous
# data type in ['object'] assume to be categorical
# if categorical - unique_values > 20 - this rule is pretty objective, will review later

In [ ]:
# check the skewness of conti. variables

cont_var = df_dict[(df_dict['Type']=='int64') | (df_dict['Type']=='float64')]['Variable'].unique().tolist()
cont_var

df_dict['skewness'] = np.NAN
df_dict['skewness_type'] = np.NAN
for var in cont_var:
  skew =  train[var].skew()
  df_dict.loc[df_dict[df_dict['Variable']==var].index,'skewness'] = skew
  if abs(skew) <= 1:
    df_dict.loc[df_dict[df_dict['Variable']==var].index,'skewness_type'] = 1
  print(f'{var} with skewness = {skew}')

In [ ]:
#good conitnues variable should with a skew between -1 and 1, ideally the unique values should be big

In [ ]:
cont_var_good = df_dict[(df_dict['skewness_type']==1) & (df_dict['Unique_Values'] > 20)]['Variable'].unique().tolist()
cont_var_good

In [ ]:
#catigorical variables
cat_var = df_dict[(df_dict['Type']=='object')|((df_dict['Type']=='int64') & (df_dict['Unique_Values'] <= 20))]['Variable'].unique().tolist()
cat_var

In [ ]:
try:
  del var_dt_summary
except:
  print('no var_dt_sumary created yet')
for i, var in enumerate(cat_var):
  #print(var)
  var_dt = train.groupby(var)[var].count().to_frame()
  var_dt.columns = ['Count']
  var_dt = var_dt.reset_index()
  var_dt.columns = ['Value','Count']
  var_dt['Variable'] = var

  try:
    var_dt_summary=pd.concat([var_dt_summary,var_dt], axis=0)
  except:
    var_dt_summary=var_dt

In [ ]:
df_dict

In [ ]:
var_dt_summary[['Variable','Value','Count']]

In [ ]:
high_card_cutoff = 13

cat_var_high_card = df_dict[(df_dict['Variable'].isin(cat_var)) & (df_dict['Unique_Values']>= high_card_cutoff)]['Variable'].unique().tolist()
cat_var_high_card

In [ ]:
imbalance_cutoff = 0.7 #if a count of a variable > 70% of total then it's inbalance

cat_var_imb = var_dt_summary[var_dt_summary['Count']> (imbalance_cutoff * df_shape[0])]['Variable'].unique().tolist()
cat_var_imb

In [ ]:
# for model only use balanced & low-card categorical variables

In [ ]:
cat_var_good = [item for item in cat_var if (item not in cat_var_high_card) and (item not in cat_var_imb)]
cat_var_good

In [ ]:
# Subset all categorical variables
def convert_cat_vars(df,cat_var_good):
   df_categorical = df[cat_var_good]
   # convert into dummies
   df_dummies = pd.get_dummies(df_categorical, columns = cat_var_good , drop_first=True)

   return df_dummies


In [ ]:
train_cat = convert_cat_vars(train,cat_var_good)
train_cat

In [ ]:
test_cat = convert_cat_vars(test,cat_var_good)
test_cat

Scale and winsorize data

In [ ]:

def convert_cont_vars(df,cat_var_good):
   df_cont = df[cont_var_good]
   # convert into dummies
   df_wzr = df_cont.clip(upper=df_cont.quantile(0.99),axis=1)
   scaler = StandardScaler()
   df_scaled = pd.DataFrame(scaler.fit_transform(df_wzr), columns=df_wzr.columns, index=df_wzr.index)

   return df_scaled

In [ ]:
train_cont = convert_cont_vars(train,cont_var_good)
train_cont

In [ ]:
test_cont = convert_cont_vars(test,cont_var_good)
test_cont

In [ ]:
train_new = pd.concat([train_cat,train_cont], axis = 1)
train_new

In [ ]:
test_new = pd.concat([test_cat,test_cont], axis = 1)
test_new

In [ ]:
train_y_log = np.log(train_y)
train_y_log

In [ ]:
# make sure all columns available in test
common_columns = test_new.columns.intersection(train_new.columns)

In [ ]:
elastic_net_cv = ElasticNetCV(cv=10, random_state=42)

# Fit the model on training data
elastic_net_cv.fit(train_new[common_columns], train_y_log)

# Best alpha and l1_ratio found via cross-validation
print(f"Best alpha: {elastic_net_cv.alpha_}")
print(f"Best l1_ratio: {elastic_net_cv.l1_ratio_}")

In [ ]:
# Make predictions on test data
y_pred_cv = elastic_net_cv.predict(test_new[common_columns])

In [ ]:


# Check model performance
mse_cv = mean_squared_error(np.log(test_y.set_index('PID')), y_pred_cv)
mse_cv

##**Zilal Section Part 1 Using Regression Model**


# **Model 1: linear regression models with penalty**

### **Feature Engineering By Zilal**
-  Replace Missing values with zeros
- Categorical Vars: Resource: https://campuswire.com/c/GB46E5679/feed/328
- Drop unnecessary features:
Resource: https://campuswire.com/c/GB46E5679/feed/328
- winsorization: Resource: https://campuswire.com/c/GB46E5679/feed/328



In [ ]:
#### Global Vars ######
# List of features to winsorize
winsorize_features = [
    "Lot_Frontage", "Lot_Area", "Mas_Vnr_Area", "BsmtFin_SF_2",
    "Bsmt_Unf_SF", "Total_Bsmt_SF", "Second_Flr_SF", "First_Flr_SF",
    "Gr_Liv_Area", "Garage_Area", "Wood_Deck_SF", "Open_Porch_SF",
    "Enclosed_Porch", "Three_season_porch", "Screen_Porch", "Misc_Val"
]



In [ ]:
# Dictionary to store running times for fold and model
running_times_models = {'Elastic_net_cv': [], 'XGboost': []}


In [ ]:

# Apply the same 95th percentiles to both training and test data
def winsorize(df, percentiles):
    for col in winsorize_features:
        df[col] = np.where(df[col] > percentiles[col], percentiles[col], df[col])
    return df



In [ ]:
# Replace missing values with zero
def zero_missing_values(df):
    # df_Of_Garage_Yr_Blt = df.loc[:,['Garage_Yr_Blt']]
    # Replace missing values with 0
    df_no_nan = df.fillna(0)
    return df_no_nan

In [ ]:
# Process the training data
def preprocess_train_data(train_file):
     # Log scale for target variable
    y_train = np.log(train_file['Sale_Price'])


    # Variables to Remove from X_train
    X_train = train_file.drop(columns=['PID', 'Street','Utilities',
                                         'Condition_2', 'Roof_Matl',
                                          'Heating','Pool_QC', 'Misc_Feature',
                                          'Low_Qual_Fin_SF', 'Pool_Area',
                                          'Longitude','Latitude',
                                          'Sale_Price'])

    # Apply winsorize_data
    # X_train = winsorize_data_2(X_train, winsorize_features,quantile=0.95)
    percentiles = {}
    for col in winsorize_features:
        percentiles[col] = np.percentile(X_train[col], 95)

    X_train = winsorize(X_train, percentiles)


    # Missing values in X_train
    X_train = zero_missing_values(X_train)

    # Convert Categorical Vars
    X_train = pd.get_dummies(X_train, drop_first=True)  # One-hot encoding for categorical variables

    # Preprocess: Standardizing numerical features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    return X_train_scaled,X_train, y_train, scaler, percentiles




In [ ]:
# Build the Elastic_Net Model
def train_model(X_train_scaled, y_train):
    # Define alphas to test
    # alphas = np.logspace(8, -2, 100)
    # Fit Elastic Net with cross-validation
    elastic_net_cv = ElasticNetCV(alphas=[0.02],
                         l1_ratio= 0.1)

    model= elastic_net_cv.fit(X_train_scaled, y_train)

    # Best alpha
    best_alpha = elastic_net_cv.alpha_
    # print(f"Best alpha: {best_alpha}")
    return model






In [ ]:
# Process the test data
def preprocess_test_data(test_file, X_train ,scaler,percentiles):
    # Variables to Remove from X_test
    X_test = test_file.drop(columns=['PID', 'Street','Utilities',
                                         'Condition_2', 'Roof_Matl',
                                          'Heating','Pool_QC', 'Misc_Feature',
                                          'Low_Qual_Fin_SF', 'Pool_Area',
                                          'Longitude','Latitude'])


    # Apply winsorize_data
    X_test = winsorize(X_test, percentiles)

    # Remove Missing value in X_test
    X_test= zero_missing_values(X_test)

    # Remove Categorical vars from X_test
    X_test = pd.get_dummies(X_test, drop_first=True)
    # Ensure the columns in the test set match the training set
    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)


    # Scale X_test
    X_test_scaled = scaler.transform(X_test)

    return X_test_scaled

In [ ]:
 # Make predictions on the test data
def make_predictions(model, X_test_scaled):
    y_pred_log = model.predict(X_test_scaled)
    # Convert back from log scale to actual prices
    y_pred = np.exp(y_pred_log)
    return y_pred,y_pred_log


In [ ]:
# Main code
def main():
    ElasticNetCV_errors=[]

    # Loop over each fold
    for fold in range(1, 11):
        # Start timer
        start_time_ten_folds = time.time()
        train_data = pd.read_csv(f'/content/drive/MyDrive/proj1/fold{fold}/train.csv')
        test_data = pd.read_csv(f'/content/drive/MyDrive/proj1/fold{fold}/test.csv')

        # Step 1: Preprocess the training data
        # X_train_scaled, X_train, y_train, scaler = preprocess_train_data(f'/content/drive/MyDrive/proj1/fold2/train.csv')
        X_train_scaled, X_train, y_train, scaler,percentiles = preprocess_train_data(train_data)

        # Step 1: Train the ElasticNet model
          # Track time for ElasticNet
        start_time_training_model = time.time()
        model = train_model(X_train_scaled, y_train)

        # Step 2: Preprocess the test data
        # X_test_scaled, X_test = preprocess_test_data(f'/content/drive/MyDrive/proj1/fold2/test.csv', X_train, scaler)
        X_test_scaled = preprocess_test_data(test_data, X_train, scaler,percentiles)

        # Step 2: Generate predictions for test data
        y_pred,y_pred_log = make_predictions(model, X_test_scaled)
        end_time_training_model = time.time()
        running_times_models['Elastic_net_cv'].append(end_time_training_model - start_time_training_model)

        y_test = pd.read_csv(f'/content/drive/MyDrive/proj1/fold{fold}/test_y.csv')

        # Step 2: Save predictions to mysubmission1.txt
        submission = pd.DataFrame({
            'PID': test_data['PID'],
            'Sale_Price': y_pred
        })

        submission.to_csv(f'/content/drive/MyDrive/proj1/fold{fold}/mysubmission1.txt', index=False)

        end_time_ten_folds = time.time()  # End timer
        print(f"Fold {fold}: Time taken = {end_time_ten_folds - start_time_ten_folds} seconds")


        # Evaluation. #### To be removed before submission####
        # Calculate Mean Squared Error first
        # Load the y_test data

        # y_test = pd.read_csv(f'/content/drive/MyDrive/proj1/fold1/test_y.csv')
        y_test['Sale_Price_Log'] = np.log(y_test['Sale_Price'])

        mse = mean_squared_error(y_test['Sale_Price_Log'], y_pred_log)

        # Calculate Root Mean Squared Error
        rmse = np.sqrt(mse)
        print(f'RMSE is {rmse} in fold {fold} with size {len(ElasticNetCV_errors)}')
        ElasticNetCV_errors.append(rmse)
        # Define the RMSE benchmark for the first 5 split
        benchmark_first_5 = 0.125
        # Define the RMSE benchmark for the rest split
        benchmark_second_5 = 0.135
        # Check if the RMSE is below the benchmark
        if fold <= 5:
          if rmse < benchmark_first_5:
            print(f"In fold - {fold} the model performs well. RMSE of {rmse:.4f} is below the benchmark of {benchmark_first_5}.")
          else:
            print(f"In the fold - {fold} the model does not meet the benchmark. RMSE of {rmse:.4f} exceeds the benchmark of {benchmark_first_5}.")

        elif fold > 5 :
          if rmse < benchmark_second_5:
             print(f"In the fold - {fold} the  model performs well. RMSE of {rmse:.4f} is below the benchmark of {benchmark_second_5}.")
          else:
             print(f"In the fold - {fold} the model does not meet the benchmark. RMSE of {rmse:.4f} exceeds the benchmark of {benchmark_second_5}.")

          ### Accuracy Table ####
      # Table for Accuracy of ElasticNetCV error
    accuracy_table = pd.DataFrame({
        'Fold': range(1, 11),
        'ElasticNetCV Model Error': ElasticNetCV_errors,
    })
    # Plot table
    fig, ax = plt.subplots(figsize=(8, 4))  # Adjust size if needed
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=accuracy_table.values, colLabels=accuracy_table.columns, loc='center')

    # Save the table as an image
    plt.savefig('accuracy_table.png', dpi=300)

    # Display the table
    plt.show()

        # Output the running times for each model and fold
    for model, times in running_times_models.items():
        for fold, time_taken in enumerate(times, start=1):
            print(f"Model: {model}, Fold: {fold}, Time: {time_taken:.2f} seconds")




if __name__ == "__main__":
  main()
